# Data retrival

In [1]:
%load_ext autotime
%load_ext autoreload
%autoreload 2

In [2]:
from datetime import datetime
startTime = datetime.now()

time: 6.97 ms


In [3]:
import myfunctions as f
print f.today()
print ""
# python version
# import sys
# print "system_info: %s"%sys.version
# current working directory
# import os
# print "path_info: %s"%os.getcwd() 

20180403

time: 8.76 ms


In [4]:
# setup elastic search
# f.initialise_es(1)
foodpanda, deliveroo, wte, websites = f.delivery_para()

time: 6.92 ms


** Retriving all food items **

In [ ]:
doc_type = "food"

time: 6.64 ms


In [ ]:
website = websites[0]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

In [ ]:
website = websites[1]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

In [ ]:
website = websites[2]
f.initialise_es(0)
website.update({"all_rec_"+doc_type : f.retrive_from_es(website, doc_type)})

** Data: keep all index and dates **

In [ ]:
### fomulate location
def fomulate_loc(w):
    """formulate storage location according to es data"""
    w['all_rec_food']["loc"] = w['all_rec_food']["_index"] + "/" + \
    w["food"] + "/" + \
    w['all_rec_food']["_id"]

def fomulate_timestamp(w):
    if w["_index"] == 'what_to_eat':
        w['all_rec_food']["timestamp"] = w['all_rec_food']['_source.startTimestampGMT']
    else:
        d = f.retrive_from_es(w, "crawling_cycle")
        cycle_time = d.set_index(w['cycle_id']).to_dict()[w['cycleStart']]
        # remove na for food names
        w['all_rec_food'] = w['all_rec_food'][~w['all_rec_food'][w["food_name"]].isnull()]
        # assign time according to food cycle
        w['all_rec_food']["timestamp"] = w['all_rec_food'][w['food_cycle']].apply(lambda s: cycle_time[s])

In [ ]:
for w in websites:
    fomulate_loc(w)     
    fomulate_timestamp(w) 
    print "%s:: number of records: %d"%(w["_index"],
                                        w['all_rec_food'].shape[0])

** Save food data with all details **

In [ ]:
import pandas as pd
df1 = pd.concat(w['all_rec_food'] for w in websites)
# reset index after concat
df1.reset_index(inplace=True, drop=True)

In [ ]:
print df1.shape
print df1.columns.tolist()
df1.sample(2,random_state=2008)

# Restaurant entities

In [ ]:
%%time
restaurant_details = f.get_restaurant_details(df1)

In [ ]:
restaurant_details.sample(2,random_state=2008)

In [ ]:
%%time
# merge records based on name
tup = lambda g: tuple(g)
unq = lambda g:  sorted(list(set(g)))
fn = {"loc":tup, "timestamp":tup, "address":unq, "neighbourhood":unq, "opening":unq,
     "phone":unq, "rating":unq, "tag":unq}
restaurant = restaurant_details.groupby("name").agg(fn)
# free the "name" column
restaurant.reset_index(inplace=True)
# update index to be "delivery_"+xxx
restaurant.reset_index(inplace=True)
restaurant["index"] = "delivery_"+restaurant["index"].apply(str)
restaurant.set_index("index", inplace=True)

In [ ]:
print restaurant.shape
print restaurant.columns.tolist()
restaurant.sample(2, random_state=2008)

In [ ]:
## save to json
file_name = "__".join(["restaurant_entities", f.today()]) + ".json"
restaurant.to_json(file_name, orient='index')
print "saved: %s"%file_name

In [ ]:
# ref dict: restaurant name : restaurant index
restaurant.reset_index(inplace=True)
restaurant_ref = restaurant.set_index(["name"]).to_dict()["index"]

# Food entities

** clean name of the food **

In [ ]:
%%time
food_details = f.get_food_details(df1, restaurant_ref)

In [ ]:
print food_details.shape
print food_details.columns.tolist()
food_details.sample(2, random_state=2008)

In [ ]:
# ref dict: food_name: clean_name
food_names = set(food_details["food_name"].tolist())
print "with %d unique menu names in %d records"%(len(food_names), food_details.shape[0])
clean_food_ref = {s:f.clean_name_v2(s) for s in food_names}

In [ ]:
%%time
# merge records based on name
tup = lambda g: tuple(g)
unq = lambda g:  sorted(list(set(g)))
fn = {"food_name":f.count_freq, "vendors": unq, "desc":unq, "price":tup, 
      "tag":unq, "loc":unq, "timestamp":unq}
food_details["clean_name"] = food_details["food_name"].apply(lambda s: clean_food_ref[s])
food_items = food_details.groupby("clean_name").agg(fn)
# free the "clean_name" column
food_items.reset_index(inplace=True)
# update index to be "food_"+xxx
food_items.reset_index(inplace=True)
food_items["index"] = "food_"+food_items["index"].apply(str)
food_items.set_index("index", inplace=True)

In [ ]:
print food_items.shape
print food_items.columns.tolist()
food_items.sample(2, random_state=2008)

In [ ]:
## save to p file
file_name = "__".join(["food_entities", f.today()]) + ".p"
f.save_file(food_items, file_name)

** construct test case **

In [ ]:
# save test case to csv
file_name = "001_entity_identification_test_case_" + f.today()+".csv"
test_case = food_items.sample(100, random_state=2008)[["food_name", "clean_name"]]
test_case["food_name"] = test_case["food_name"].apply(lambda s: s[0][0])
test_case.to_csv(file_name,index=False)
print "saved: %s"%file_name

** total time **

In [ ]:
print datetime.now() - startTime 